In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

# ── CONFIG ──────────────────────────────────────────────────────────
user     = 'etl_user1'
password = quote_plus('3December2000@')  # encode '@' as '%40'
host     = '127.0.0.1'
port     = '5432'
db       = 'etl_demo'

engine = create_engine(
    f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'
)

# ── 1) Read raw data ─────────────────────────────────────────────────
sql_raw = "SELECT order_date, sales, discount, profit, order_id FROM public.sales_raw;"
df_raw = pd.read_sql(sql_raw, engine, parse_dates=['order_date'])

# ── 2) Aggregate by year & month ────────────────────────────────────
df_raw['year']  = df_raw['order_date'].dt.year
df_raw['month'] = df_raw['order_date'].dt.month

summary = (
    df_raw
    .groupby(['year','month'])
    .agg(
      total_sales   = ('sales',    'sum'),
      total_orders  = ('order_id','nunique'),
      avg_discount  = ('discount','mean'),
      total_profit  = ('profit',   'sum')
    )
    .reset_index()
)

# ── 3) Write into sales_summary ─────────────────────────────────────
with engine.begin() as conn:
    # clear old data
    conn.execute(text("TRUNCATE TABLE public.sales_summary;"))
    # bulk insert
    summary.to_sql(
        'sales_summary',
        con=conn,
        if_exists='append',
        index=False,
        method='multi'
    )

print("✅ sales_summary updated:")
print(summary.head())


✅ sales_summary updated:
   year  month  total_sales  total_orders  avg_discount  total_profit
0  2014      1     14236.90            32      0.126582       2450.18
1  2014      2      4519.92            28      0.176087        862.30
2  2014      3     55691.04            71      0.167516        498.72
3  2014      4     28295.35            66      0.110000       3488.86
4  2014      5     23648.28            69      0.155328       2738.74
